1203. Sort Items by Groups Respecting Dependencies
Hard
Topics
premium lock icon
Companies
Hint
There are n items each belonging to zero or one of m groups where group[i] is the group that the i-th item belongs to and it's equal to -1 if the i-th item belongs to no group. The items and the groups are zero indexed. A group can have no item belonging to it.

Return a sorted list of the items such that:

The items that belong to the same group are next to each other in the sorted list.
There are some relations between these items where beforeItems[i] is a list containing all the items that should come before the i-th item in the sorted array (to the left of the i-th item).
Return any solution if there is more than one solution and return an empty list if there is no solution.

 

Example 1:



Input: n = 8, m = 2, group = [-1,-1,1,0,0,1,0,-1], beforeItems = [[],[6],[5],[6],[3,6],[],[],[]]
Output: [6,3,4,1,5,2,0,7]
Example 2:

Input: n = 8, m = 2, group = [-1,-1,1,0,0,1,0,-1], beforeItems = [[],[6],[5],[6],[3],[],[4],[]]
Output: []
Explanation: This is the same as example 1 except that 4 needs to be before 6 in the sorted list.
 

Constraints:

1 <= m <= n <= 3 * 104
group.length == beforeItems.length == n
-1 <= group[i] <= m - 1
0 <= beforeItems[i].length <= n - 1
0 <= beforeItems[i][j] <= n - 1
i != beforeItems[i][j]
beforeItems[i] does not contain duplicates elements.

In [ ]:
class Solution:
    def topologicalSort(self, successors: List[List[int]], predecessors_count: List[int], num_nodes: int) -> List[int]:
        order = []
        nodes_with_no_predecessors = deque(node for node in range(num_nodes) if not predecessors_count[node])
        
        while nodes_with_no_predecessors:
            node = nodes_with_no_predecessors.popleft()
            order.append(node)
            for successor in successors[node]:
                predecessors_count[successor] -= 1
                if not predecessors_count[successor]:
                    nodes_with_no_predecessors.append(successor)
        
        return order if len(order) == num_nodes else []

    def sortItems(self, n: int, m: int, group: List[int], beforeItems: List[List[int]]) -> List[int]:
        for item in range(n):
            if group[item] == -1: 
                group[item] = m
                m += 1
        
        successors_group, successors_item = [[] for _ in range(m)], [[] for _ in range(n)]
        predecessors_count_group, predecessors_count_item = [0] * m, [0] * n
        
        for item in range(n):
            current_group = group[item]
            for before in beforeItems[item]:
                before_group = group[before]
                if current_group == before_group:
                    successors_item[before].append(item)
                    predecessors_count_item[item] += 1
                else:
                    successors_group[before_group].append(current_group)
                    predecessors_count_group[current_group] += 1
        
        groups_order = self.topologicalSort(successors_group, predecessors_count_group, m)
        items_order = self.topologicalSort(successors_item, predecessors_count_item, n)
        
        if not groups_order or not items_order:
            return []
        
        items_grouped = [[] for _ in range(m)]
        for item in items_order:
            items_grouped[group[item]].append(item)
        
        result = []
        for grp in groups_order:
            result.extend(items_grouped[grp])
        
        return result

In [ ]:
class Solution:
    def sortItems(self, n: int, m: int, group: List[int], beforeItems: List[List[int]]) -> List[int]:
        
        def topologicalSort(nodes, pred, succ):
            order, no_pred = [], deque(node for node in nodes if not pred[node])
            while no_pred:
                node = no_pred.popleft()
                order.append(node)
                for s in succ[node]:
                    pred[s].discard(node)
                    if not pred[s]:
                        no_pred.append(s)
            return order if len(order) == len(nodes) else []
        
        group_to_items, new_group = defaultdict(set), m
        for item in range(n):
            if group[item] == -1: group[item] = new_group; new_group += 1
            group_to_items[group[item]].add(item)
        
        intra_pred, intra_succ, inter_pred, inter_succ = defaultdict(set), defaultdict(set), defaultdict(set), defaultdict(set)
        for item in range(n):
            for before in beforeItems[item]:
                if group[item] == group[before]:
                    intra_pred[item].add(before)
                    intra_succ[before].add(item)
                else:
                    inter_pred[group[item]].add(group[before])
                    inter_succ[group[before]].add(group[item])
        
        groups_order = topologicalSort(list(group_to_items.keys()), inter_pred, inter_succ)
        if not groups_order: return []
        
        items_order = []
        for grp in groups_order:
            order = topologicalSort(group_to_items[grp], intra_pred, intra_succ)
            if not order: return []
            items_order.extend(order)
        
        return items_order

In [ ]:
class Solution:
    def sortItems(self, n: int, m: int, group: List[int], beforeItems: List[List[int]]) -> List[int]:
        def dfs_diff_group(group):
            if group_visited[group] == 1:
                return True
            if group_visited[group] == 2:
                return False
            group_visited[group] = 1
            for item in group_to_items[group]:
                for nxt_item in beforeItems[item]:
                    if item_to_group[item] != item_to_group[nxt_item]:
                        if dfs_diff_group(item_to_group[nxt_item]):
                            return True
            group_visited[group] = 2
            for item in group_to_items[group]:
                if dfs_same_group(item):
                    return True
            return False

        def dfs_same_group(item):
            if visited[item] == 1:
                return True
            if visited[item] == 2:
                return False
            visited[item] = 1
            for nxt_item in beforeItems[item]:
                if item_to_group[item] == item_to_group[nxt_item]:
                    if dfs_same_group(nxt_item):
                        return True
            visited[item] = 2
            ans.append(item)
            return False
        
        item_to_group = list(group)
        group_to_items = defaultdict(list)
        num_groups = m
        for i, g in enumerate(item_to_group):
            if g == -1:
                item_to_group[i] = num_groups
                group_to_items[num_groups].append(i)
                num_groups += 1
            else:
                group_to_items[g].append(i)

        # 0 is unvisited
        # 1 is processing
        # 2 is visited
        visited = [0] * n 
        group_visited = [0] * num_groups
        ans = []
        for g in range(num_groups):
            if dfs_diff_group(g):
                return []
        return ans